In [1]:
import sys
sys.path.insert(0, '..')

from core.web_discovery import WebDiscovery
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()

# Initialize web discovery
discovery = WebDiscovery()
sources_dir = Path('../data/sources')
sources_dir.mkdir(parents=True, exist_ok=True)

print("✓ Web Discovery initialized")


✓ Web Discovery initialized


## Step 1: Enter Research Topic

What topic are you researching?

In [ ]:
# # Option 1: Enter your research topic manually
# research_topic = "Knowledge graphs and semantic web technologies"

# Option 2: Extract research questions from a DOCX file
from core.rag_engine import VaultRAG
from pathlib import Path

docx_path = Path("../data/sources/LinkedIn+Blog_Posts_Set_1_of_3_EU_Data_Act_Linked_Data.docx")

# print(f"📄 Processing: {docx_path.name}")

# Initialize RAG to convert DOCX to TXT
rag = VaultRAG(sources_dir="../data/sources", verbose=True)

# Convert DOCX to TXT
txt_path = rag._convert_docx_to_txt(docx_path)

if txt_path and txt_path.exists():
    print(f"✓ Converted to: {txt_path.name}")
    
    # Read the TXT file
    with open(txt_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    print(f"📊 Content length: {len(content)} characters")
    print(f"\n📖 First 500 characters:\n{content[:500]}\n")
    
    if len(content.strip()) < 100:
        print("⚠️ Warning: Content too short. Using fallback topic.")
        research_topic = "EU Data Act and Linked Data governance"
    else:
        # Use AI to extract main research questions
        from openai import OpenAI
        client = OpenAI()
        
        # Use first 5000 characters for extraction
        content_sample = content[:5000]
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a research assistant. Extract the main research questions and topics from the given document. Return a concise research topic statement (one sentence) that captures the core themes."},
                {"role": "user", "content": f"Extract the main research topic from this document:\n\n{content_sample}"}
            ],
            temperature=0.3
        )
        
        research_topic = response.choices[0].message.content.strip()
else:
    print("✗ Failed to convert DOCX. Using fallback topic.")
    research_topic = "EU Data Act and Linked Data governance"

print(f"\n🎯 Research topic: {research_topic}")


📄 Processing: LinkedIn+Blog_Posts_Set_1_of_3_EU_Data_Act_Linked_Data.docx
Loading documents from ..\data\sources...
Found 19 files
  ✓ Loaded: Understanding the Themes and Concepts in the Knowledge Graph
  ✓ Loaded: Understanding the Themes and Concepts in the Knowledge Graph
  ✓ Loaded: YouTube-8nbb6CwpPMA
  ✓ Loaded: YouTube Links to Process
  ✓ Loaded: RDF Levels the Advantages of Labeled Property Graphs & Keeps Three Key Benefits_ Standards, Semantics & Interoperability _ Ontotext_files\f(1).txt
  ✓ Loaded: RDF Levels the Advantages of Labeled Property Graphs & Keeps Three Key Benefits_ Standards, Semantics & Interoperability _ Ontotext_files\f(2).txt
  ✓ Loaded: RDF Levels the Advantages of Labeled Property Graphs & Keeps Three Key Benefits_ Standards, Semantics & Interoperability _ Ontotext_files\f.txt
  ✓ Loaded: Cloud and Edge Computing_ a different way of using IT — Brochure _ Shaping Europe’s digital future.pdf
  ✓ Loaded: Cloud switching under the EU Data Act.pdf
  ✓ Loaded:

## Step 2: Generate Search Queries

AI will generate optimized search queries for your topic.

In [ ]:
# Option 1: Manual search (generate queries, you search manually)
# Uncomment to use:
"""
queries = discovery._generate_search_queries(research_topic)

print("\n📝 Generated Search Queries:\n")
for i, query in enumerate(queries, 1):
    print(f"{i}. {query}")

print("\n💡 Copy these queries and search on:")
print("   - Google Scholar")
print("   - arXiv")
print("   - Medium")
print("   - Academic journals")
"""

# Option 2: Automated search (uses free APIs)
print("🔍 Searching for articles automatically...\n")

# Generate search queries
queries = discovery._generate_search_queries(research_topic)
print(f"📝 Generated {len(queries)} search queries\n")

# Search using free APIs (arXiv + Semantic Scholar)
all_results = []

for i, query in enumerate(queries, 1):
    print(f"[{i}/{len(queries)}] Searching: {query}")
    
    # Search academic sources (FREE - no API key needed)
    results = discovery.search_web(query, max_results=5, source='all')
    
    print(f"  ✓ Found {len(results)} results")
    all_results.extend(results)

print(f"\n✅ Total results found: {len(all_results)}")

# Display results with indices
print("\n📚 Search Results:\n")
for i, result in enumerate(all_results[:20], 1):  # Show first 20
    print(f"[{i}] {result['title']}")
    print(f"    Source: {result['source']}")
    print(f"    URL: {result['url']}")
    print(f"    Snippet: {result['snippet'][:150]}...")
    print()

print(f"\n💡 Next step: Copy the URLs you want to extract to Step 3 below.")
print(f"   Or manually add more URLs from Google Scholar, Medium, etc.")


📝 Generated Search Queries:

1. 1. "impact of EU Data Act on data ownership and accessibility for individuals and businesses"
2. 2. "limitations of traditional cloud management systems in compliance with EU Data Act"
3. 3. "transitioning from traditional cloud infrastructure to Linked Data platforms for EU Data Act compliance"
4. 4. "semantic interoperability and the EU Data Act implications for data sharing"
5. 5. "EU Data Act and its effects on data accessibility in public organizations"

💡 Copy these queries and search on:
   - Google Scholar
   - arXiv
   - Medium
   - Academic journals


## Step 3: Paste URLs

After searching, paste the URLs you want to extract (one per line).

In [ ]:
# Option A: Use automated search results
# Select which search results to extract (by index number from above)
selected_indices = [1, 2, 3, 4, 5]  # Change these to select specific results

# Extract URLs from selected results
urls = [all_results[i-1]['url'] for i in selected_indices if i <= len(all_results)]

print(f"📋 Selected {len(urls)} URLs from search results:\n")
for i, url in enumerate(urls, 1):
    print(f"{i}. {url}")

# Option B: Manually paste URLs (uncomment to use)
"""
urls_text = '''
https://example.com/article1
https://example.com/article2
https://example.com/article3
'''

urls = [url.strip() for url in urls_text.strip().split('\n') if url.strip()]

print(f"\n📋 Found {len(urls)} URLs to process")
for i, url in enumerate(urls, 1):
    print(f"{i}. {url}")
"""

## Step 4: Extract Articles

Extract content from URLs and preview quality.

In [ ]:
# Extract articles
articles = []

print("\n🔍 Extracting articles...\n")

for i, url in enumerate(urls, 1):
    print(f"[{i}/{len(urls)}] Processing: {url}")
    
    try:
        article = discovery.extract_article(url)
        
        if article:
            # Get quality assessment
            assessment = discovery.assess_quality(article)
            article['assessment'] = assessment
            articles.append(article)
            
            print(f"  ✓ {article['title']}")
            print(f"    Author: {article.get('author', 'Unknown')}")
            print(f"    Length: {len(article['content'])} chars")
            print(f"    Quality: {assessment.get('quality_score', 'N/A')}")
        else:
            print(f"  ✗ Could not extract article")
    
    except Exception as e:
        print(f"  ✗ Error: {str(e)}")
    
    print()

print(f"\n✓ Successfully extracted {len(articles)} articles")

## Step 5: Review and Filter

Review extracted articles and select which ones to save.

In [ ]:
# Display articles with indices
print("\n📚 Extracted Articles:\n")

for i, article in enumerate(articles):
    print(f"[{i}] {article['title']}")
    print(f"    URL: {article['url']}")
    print(f"    Author: {article.get('author', 'Unknown')}")
    print(f"    Length: {len(article['content'])} characters")
    
    # Show assessment if available
    if 'assessment' in article:
        assessment = article['assessment']
        print(f"    Quality: {assessment.get('quality_score', 'N/A')}")
    
    print()

print("\n💡 To save specific articles, set indices_to_save below.")
print("   Example: indices_to_save = [0, 1, 3]  # Save articles 0, 1, and 3")
print("   Or leave empty to save all: indices_to_save = []")

In [ ]:
# Select which articles to save (empty list = save all)
indices_to_save = []  # Change this to select specific articles

# If empty, save all
if not indices_to_save:
    articles_to_save = articles
else:
    articles_to_save = [articles[i] for i in indices_to_save if i < len(articles)]

print(f"Selected {len(articles_to_save)} articles to save")

## Step 6: Save to Sources Directory

Save selected articles as markdown files in `data/sources/`.

In [ ]:
# Save articles
import re
from datetime import datetime

saved_files = []

print("\n💾 Saving articles...\n")

for article in articles_to_save:
    # Create safe filename
    safe_title = re.sub(r'[^\w\s-]', '', article['title'])
    safe_title = re.sub(r'[-\s]+', '-', safe_title)[:100]
    filename = f"{safe_title}.md"
    filepath = sources_dir / filename
    
    # Create markdown content with frontmatter
    content = f"""---
title: {article['title']}
author: {article.get('author', 'Unknown')}
url: {article['url']}
date_extracted: {datetime.now().strftime('%Y-%m-%d')}
tags: [web-article, {research_topic.lower().replace(' ', '-')}]
---

# {article['title']}

**Author:** {article.get('author', 'Unknown')}  
**Source:** {article['url']}  
**Extracted:** {datetime.now().strftime('%Y-%m-%d')}

---

{article['content']}
"""
    
    # Write file
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(content)
    
    saved_files.append(filepath)
    print(f"  ✓ Saved: {filename}")

print(f"\n✅ Successfully saved {len(saved_files)} articles to data/sources/")
print("\n🔄 Next steps:")
print("   1. Launch the UI: python server.py")
print("   2. Ask questions about your new sources!")